## Урок 1

### Практическое задание:

In [1]:
import pandas as pd
import numpy as np

#### 1. Сравните метрики hit_rate@k, precision@k. <br>Какую метрику использовать предпочтительно и почему? <br>Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


Метрика Hit_rate@k принемает всего два значения:<br>"1" - если хотя бы один релевантный (купленный) товар был среди топ-k рекомендованных<br>и<br>"0" - если ни одного релевантного (купленного) товара не было среди топ-k рекомендованных<br>Метрику Hit_rate@k можно интерпретировать как - "был ли хотя бы один релевантный товар среди топ-k рекомендованных"

Метрика Precision@k показывает долю релевантных товаров среди топ-k рекомендованных.<br>Метрику Precision@k можно интерпретировать как - "точность наших топ-k рекомендованных товаров"

Для большенства задач предпочтительно использовать метрику Precision@k нежели метрику Hit_rate@k, потому что метрика Precision@k, в отличие от метрики Hit_rate@k, дает более полную информацию о точности наших рекомендаций, позволяет оценить качество нашей рекомендательной модели и при необходимости настраивать модель для улучшения этой метрики. 

Но есть ряд задач, где уместно использовать Hit_rate. Это рекомендация, что называется "штучного товара", таких как машины, квартиры, ювелирные изделия (не в плане украшений, а в плане инвестиций).

#### 2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



Использование логарифма можно объяснить следующими интуитивными соображениями: с точки зрения ранжирования позиции в начале списка отличаются гораздо сильнее, чем позиции в его конце. Так, в случае поискового движка между позициями 1 и 11 целая пропасть (лишь в нескольких случаях из ста пользователь заходит дальшей первой страницы поисковой выдачи), а между позициями 101 и 111 особой разницы нет — до них мало кто доходит. Эти субъективные соображения прекрасно выражаются с помощью логарифма:
$$ \frac{1}{\log_2(1+1)} - \frac{1}{\log_2(1+11)}\approx0.721,$$ a $$ \frac{1}{\log_2(1+101)} - \frac{1}{\log_2(1+111)}\approx0.003.$$

Это так же актуально для статей или постов на форумах. Свежие записи должны чаще попадать в топ.<br>Расчет рейтинга в Reddit:$$Rank=\log_{10}(max(1,U-D))-\frac{|U-D|T}{const}$$


где U = число голосов «за», D = число голосов «против», T = время записи. Первое слагаемое оценивает «качество записи», а второе делает поправку на время.

Вместо логарифма, на мой взгляд, можно иcпользовать функцию экспоненты: $$e^{(k-1)}$$

#### 3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).


$\large \ F_1 = 2 \cdot \frac{precision \cdot recall}{precision + recall}$

#### 4. boughted = [1, 3, 5, 7, 9, 11]  <br>recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  <br> Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8 <br> Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

#### IDCG должен считаться для всех 8 позиций рекомендаций

In [2]:
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]
boughted = [1, 3, 5, 7, 9, 11]

$pr@8 = \frac{0 + 0 + 1 + 1 + 1 +1}{8} = 50\%$ 

In [3]:
def precision_at_k(recommended_list, bought_list, k):
    
    bought_list = np.array(boughted)
    recommended_list = np.array(recommended)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [4]:
precision_at_k(recommended, boughted, k=8)

0.5

$rec@8 = \frac{0 + 0 + 1 + 1 + 1 +1}{6}\approx67\%$ 

In [5]:
def recall_at_k(recommended_list, bought_list, k):
    
    bought_list = np.array(boughted)
    recommended_list = np.array(recommended)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [6]:
recall_at_k(recommended, boughted, k=8)

0.6666666666666666

$ap@8=\frac{1}{8}(\frac{0}{1}+\frac{1}{2}+\frac{2}{3}+\frac{0}{4}+\frac{3}{5}+\frac{4}{6}+\frac{0}{7}+\frac{0}{8})\approx0.30417$

In [7]:
def ap_k(recommended_list, bought_list, k):
    
    bought_list = np.array(boughted)
    recommended_list = np.array(recommended)[:k]
    
    flags = np.isin(recommended_list, bought_list)
   
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

In [8]:
ap_k(recommended, boughted, k=8)

0.30416666666666664

$$NDCG = \frac{DCG}{ideal DCG}$$

$DCG@8 = \frac{1}{8}*(0 / 1 + 1 / \log_2(2) + 1 / \log_2(3) + 0 / \log_2(4) + 1 / \log_2(5) + 1 / \log_2(6) + 0 / \log_2(7) + 0 / \log_2(8))\approx0,3061$  
$ideal DCG@8 = \frac{1}{8}*((1 / 1 + 1 / \log_2(2) + 1 / \log_2(3) + 1 / \log_2(4) + 1 / \log_2(5) + 1 / \log_2(6) + 1 / \log_2(7) + 1 / \log_2(8))\approx0,5797$  

$NDCG = \frac{0,3061}{0,5797}\approx0,528$

In [9]:
def ndcg_k(recommended_list, bought_list, k):
    
    bought_list = np.array(boughted)
    recommended_list = np.array(recommended)[:k]
    
    flags = np.isin(recommended_list, bought_list)
   
    if sum(flags) == 0:
        return 0
    
    if flags[0] == True:
            sum_d = 1
    else: 
            sum_d = 0
    
    for i in range(1, k): 
        
        if flags[i] == True:
            d_ = 1 / np.log2(i + 1)
            sum_d += d_
            
    dcg_k = sum_d / len(flags)
    
    print (f"DCG_k = {dcg_k}")
    
    sum_i = 1
    
    for i in range(1, k):
        i_ = 1 / np.log2(i + 1)
        sum_i += i_
        
    i_dcg_k = sum_i / len(flags)
    
    print (f"idealDCG_k = {i_dcg_k}")
    
    result = dcg_k / i_dcg_k
    
    return print(f"NDCG_k = {result}")

In [10]:
ndcg_k(recommended, boughted, k=8)

DCG_k = 0.30605738985992403
idealDCG_k = 0.5797499549150934
NDCG_k = 0.5279127445637272


$mrr@8=\frac{1}{8}(\frac{0}{1}+\frac{1}{2}+\frac{1}{3}+\frac{0}{4}+\frac{1}{5}+\frac{1}{6}+\frac{0}{7}+\frac{0}{8})=0.15$

In [11]:
def reciprocal_rank(recommended_list, bought_list, k):
    ranks=0
    recommended_list = (recommended_list)[:k]
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

In [12]:
reciprocal_rank(recommended, boughted, k=8)

0.15